<a href="https://colab.research.google.com/github/divyadarshini11/LLM-Projects/blob/main/Fake_news_classifier_using_Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

from datasets import load_dataset

# Load the dataset from Hugging Face
dataset = load_dataset("ErfanMoosaviMonazzah/fake-news-detection-dataset-English")

# Save the train split to a CSV file
dataset["train"].to_csv("train.csv", index=False)

# Optional: Confirm it's saved
import pandas as pd
df = pd.read_csv("train.csv")
df.head()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/487 [00:00<?, ?B/s]

train.tsv:   0%|          | 0.00/78.4M [00:00<?, ?B/s]

validation.tsv:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8267 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/30 [00:00<?, ?ba/s]

,Unnamed: 0,title,text,subject,date,label
0,2619,Ex-CIA head says Trump remarks on Russia inter...,Former CIA director John Brennan on Friday cri...,politicsNews,"July 22, 2017",1
1,16043,YOU WON’T BELIEVE HIS PUNISHMENT! HISPANIC STO...,How did this man come to OWN this store? There...,Government News,"Jun 19, 2017",0
2,876,Federal Reserve governor Powell's policy views...,President Donald Trump on Thursday tapped Fede...,politicsNews,"November 2, 2017",1
3,19963,SCOUNDREL HILLARY SUPPORTER STARTS “TrumpLeaks...,Hillary Clinton ally David Brock is offering t...,left-news,"Sep 17, 2016",0
4,10783,NANCY PELOSI ARROGANTLY DISMISSES Questions on...,Pleading ignorance is a perfect ploy for Nancy...,politics,"May 26, 2017",0


In [ ]:
df.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
subject,0
date,0
label,0


In [ ]:
df.shape

(30000, 6)

In [ ]:
# Get input features except for label
X=df.drop('label',axis=1)

In [ ]:
# Output label
y=df['label']

In [ ]:
y.value_counts()

,count
label,
0,15478
1,14522


In [ ]:
import tensorflow as tf


In [ ]:
tf.__version__

'2.18.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [ ]:
# vocalbulary size
vocabulary_size = 20000

In [ ]:
messages = X.copy()

In [ ]:
messages.head(10)

,Unnamed: 0,title,text,subject,date
0,2619,Ex-CIA head says Trump remarks on Russia inter...,Former CIA director John Brennan on Friday cri...,politicsNews,"July 22, 2017"
1,16043,YOU WON’T BELIEVE HIS PUNISHMENT! HISPANIC STO...,How did this man come to OWN this store? There...,Government News,"Jun 19, 2017"
2,876,Federal Reserve governor Powell's policy views...,President Donald Trump on Thursday tapped Fede...,politicsNews,"November 2, 2017"
3,19963,SCOUNDREL HILLARY SUPPORTER STARTS “TrumpLeaks...,Hillary Clinton ally David Brock is offering t...,left-news,"Sep 17, 2016"
4,10783,NANCY PELOSI ARROGANTLY DISMISSES Questions on...,Pleading ignorance is a perfect ploy for Nancy...,politics,"May 26, 2017"
5,18522,EU's Tusk appealed to Rajoy to avoid escalatio...,European Council President Donald Tusk appeale...,worldnews,"October 2, 2017"
6,270,Country Guitarist Who Survived Vegas Shooting ...,"Caleb Keeter, a lifelong proponent of the Seco...",News,"October 2, 2017"
7,7628,Clinton says 'there is no case here' in FBI em...,Democratic presidential candidate Hillary Clin...,politicsNews,"October 31, 2016"
8,9599,ABC NEWS REPORTS: Las Vegas Massacre Suspect’s...,The investigation into the Las Vegas massacre ...,politics,"Oct 25, 2017"
9,11234,BONKERS BERNIE SANDERS: Prioritizing Jobs Over...,https://www.youtube.com/watch?v=GPqQIlWksbgVer...,politics,"Apr 1, 2017"


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['text'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

['former cia director john brennan friday critic disgrac presid donald trump effort play u intellig agenc assess russia meddl u elect trump administr dog investig alleg russian interfer last year u presidenti elect possibl tie campaign team speak one day first meet russian presid vladimir putin hamburg earlier month trump said suspect russian interfer elect one know sure type comment disgrac person said asham said brennan cia chief former presid barack obama aspen secur forum special counsel robert mueller sever u congression committe investig whether russia interf elect collud trump campaign tri swing race favor democrat rival hillari clinton probe focus almost exclus moscow action lawmak intellig offici said evid surfac publicli implic countri moscow deni interfer trump said campaign collud russia brennan said disappoint presid handl secur issu first six month offic must say disappoint see term mr trump intern stage think pose seriou question keep safe nation secur brennan said speak

In [18]:
onehot_encoding = [one_hot(words, vocabulary_size) for words in corpus]

In [22]:
sent_lenght = 20
embedded_docs = pad_sequences(onehot_encoding, padding='pre', maxlen=sent_lenght)

In [23]:
## creating model
embedding_feature = 100
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_feature, input_length=sent_lenght))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, 20))
print(model.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 20, 100)        │     2,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 200)            │       160,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,161,001 (8.24 MB)

 Trainable params: 2,161,001 (8.24 MB)

 Non-trainable params: 0 (0.00 B)

None


In [24]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [25]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [26]:
# Training Model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.8695 - loss: 0.2783 - val_accuracy: 0.9551 - val_loss: 0.1284
Epoch 2/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9810 - loss: 0.0577 - val_accuracy: 0.9541 - val_loss: 0.1408
Epoch 3/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.9918 - loss: 0.0239 - val_accuracy: 0.9557 - val_loss: 0.1993
Epoch 4/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9953 - loss: 0.0135 - val_accuracy: 0.9515 - val_loss: 0.2323
Epoch 5/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9976 - loss: 0.0073 - val_accuracy: 0.9418 - val_loss: 0.2689
Epoch 6/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9978 - loss: 0.0068 - val_accuracy: 0.9538 - val_loss: 0.2640
Epoch 7/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9986 - loss: 0.0047 - val_accuracy: 0.9519 - val_loss: 0.3143
Epoch 8/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9987 - loss: 0.0041 - val_accuracy

In [27]:
y_pred = model.predict(X_test)

310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


In [28]:
import numpy as np
y_pred = np.where(y_pred>=0.5,1,0)

In [29]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test, y_pred)

array([[4908,  247],
       [ 249, 4496]])

In [31]:
print(accuracy_score(y_test, y_pred))

0.9498989898989899
